In [1]:
import pandas as pd
import importlib
import fileDistanceProcessing as fdp
import importlib
importlib.reload(fdp) 

<module 'fileDistanceProcessing' from '/home/marko/Desktop/ip2/ip2_projekat/source/fileDistanceProcessing.py'>

In [2]:
X=pd.read_csv("../data/csv/sars2_mers_sars1.txt",skipinitialspace = True )

In [3]:
X.columns = X.columns.str.strip()
X.columns

Index(['Virus', 'ACCESSION', 'WHO_TYPE', 'ACCESSION_PROT', 'PROTEIN',
       'DUZINA_NUC', 'KOD_NUC'],
      dtype='object')

In [4]:

X.head(20)

,Virus,ACCESSION,WHO_TYPE,ACCESSION_PROT,PROTEIN,DUZINA_NUC,KOD_NUC
0,SARS_COV_2,LC636493.1,Alpha,BCY15809.1,ORF1ab polyprotein ...,21279,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
1,SARS_COV_2,LC638682.1,Alpha,BCZ16426.1,ORF1ab polyprotein ...,21279,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
2,SARS_COV_2,BS001166.1,Alpha,BDB07859.1,ORF1ab polyprotein ...,21279,ATGGAGAGCTTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
3,SARS_COV_2,BS001219.1,Alpha,BDB46179.1,ORF1ab polyprotein ...,21279,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
4,SARS_COV_2,BS001143.1,Alpha,BDA77043.1,ORF1ab polyprotein ...,21279,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
5,SARS_COV_2,BS001382.1,Alpha,BDD78230.1,ORF1a polyprotein ...,13206,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
6,SARS_COV_2,BS001191.1,Alpha,BDB08160.1,ORF1a polyprotein ...,13206,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
7,SARS_COV_2,BS001144.1,Alpha,BDA77055.1,ORF1ab polyprotein ...,21279,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
8,SARS_COV_2,BS001171.1,Alpha,BDB07919.1,ORF1ab polyprotein ...,21279,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...
9,SARS_COV_2,BS001200.1,Alpha,BDB45952.1,ORF1a polyprotein ...,13206,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [6]:
import pickle
import os
from zipfile import ZipFile

#raspakuj arhivu
with ZipFile("../data/distance_matrix.zip",'r') as zip:
    zip.extractall("..")

with open("../data/distance_matrix", "rb") as infile:
     	distances = pickle.load(infile)

os.remove("../data/distance_matrix")

In [7]:
#ista je metrika samo se sada racuna instant na osnovu indeksa instanci
def distance_from_matrix(x, y):
    
    i = int(x[-1]) #indeks je na poslednjem mestu u listi
    j = int(y[-1])

    # matrica je u obliku trougla bez dijagonale originalne matrice
    first = min(i, j)
    second = max(i, j) - (first + 1) # veci indeks smanji za broj instanci u redu matrice do dijagonale 

    return 0 if i == j else distances[first][second] #udaljenost x od y

In [8]:
X_len = X["DUZINA_NUC"].tolist()

# uzima u obzir i pocetne duzine sekvenci
def distance_with_num_triplets_difference(x, y):

    i = int(x[-1]) #indeks je na poslednjem mestu u listi
    j = int(y[-1])

    first = min(i, j)
    second = max(i, j) - (first + 1)

    # distanca se povecava za razliku u broju tripleta (kao da nema odsecacnja)
    return 0 if i == j else distances[first][second] + abs(X_len[i] - X_len[j]) / 3.0 #udaljenost x od y

In [9]:
# uzima u obzir i pocetne duzine sekvenci
def distance_with_num_nucleotides_difference(x, y):

    i = int(x[-1])
    j = int(y[-1])

    first = min(i, j)
    second = max(i, j) - (first + 1)

    # distanca se povecava za razliku u broju nukleotida (kao da nema odsecacnja)
    return 0 if i == j else distances[first][second] + abs(X_len[i] - X_len[j]) #udaljenost x od y

<h1>KNN on virus type using p-adic distance </h1>

In [10]:
y_virus=X["Virus"]
X_virus=[] # potreban nam je samo indeks

# na svaku instancu X_virus smo appendovali njen index
for i in range(len(y_virus)):
    X_virus.append([0, i]) # 0 je dodata zbog knn da ne pravi problem
    
X_train, X_test, y_train, y_test = train_test_split(X_virus, y_virus, test_size=0.33, stratify=y_virus)

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix

def perform_and_evaluate_knn(k, metric):
    model = KNeighborsClassifier(n_neighbors=k, algorithm='ball_tree', metric=metric)
    model.fit(X_train, y_train)

    y_pred=model.predict(X_test)
    print(f'model evalutaion for k={k} and metric={str(metric)}')
    print(f'accuracy_score: {accuracy_score(y_test,y_pred)}')
    print(confusion_matrix(y_test,y_pred))

In [12]:
perform_and_evaluate_knn(3, distance_from_matrix)

model evalutaion for k=3 and metric=<function distance_from_matrix at 0x7f36978af4c0>
accuracy_score: 0.9967867798944228
[[ 609    0    2]
 [   3    0    3]
 [   6    0 3734]]


In [13]:
perform_and_evaluate_knn(3, distance_with_num_triplets_difference)

model evalutaion for k=3 and metric=<function distance_with_num_triplets_difference at 0x7f36978af940>
accuracy_score: 0.9986229056690383
[[ 611    0    0]
 [   1    0    5]
 [   0    0 3740]]


In [14]:
perform_and_evaluate_knn(3, distance_with_num_nucleotides_difference)

model evalutaion for k=3 and metric=<function distance_with_num_nucleotides_difference at 0x7f36978afe50>
accuracy_score: 0.9986229056690383
[[ 611    0    0]
 [   1    0    5]
 [   0    0 3740]]


<h1>KNN on protein using p-adic distance </h1>

In [15]:
X.columns

Index(['Virus', 'ACCESSION', 'WHO_TYPE', 'ACCESSION_PROT', 'PROTEIN',
       'DUZINA_NUC', 'KOD_NUC'],
      dtype='object')

In [16]:
y_protein=X["PROTEIN"].str.strip()
X_protein=X_virus
X_train, X_test, y_train, y_test = train_test_split(X_virus, y_protein, test_size=0.33, stratify=y_protein)

In [17]:
y_protein.unique()

array(['ORF1ab polyprotein', 'ORF1a polyprotein', 'surface glycoprotein',
       'ORF7a protein', 'ORF8 protein', 'ORF3a protein',
       'nucleocapsid phosphoprotein', 'ORF7b protein',
       'membrane glycoprotein', 'envelope protein', 'ORF10 protein',
       'ORF6 protein', 'ORF3 protein', 'ORF4a protein', 'ORF5 protein',
       'ORF1b polyprotein', 'ORF4b protein', 'ORF8b protein'],
      dtype=object)

In [18]:
perform_and_evaluate_knn(3, distance_from_matrix)

model evalutaion for k=3 and metric=<function distance_from_matrix at 0x7f36978af4c0>
accuracy_score: 0.6821207252696809
[[   0    0    0    0    1    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    4 1367    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    8 1891    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    2    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0   33    0    2    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0   73    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0   22    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0   42    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0    2   31    0    0    0    0    0
     0    0    1    0]
 [   0    0    0    0   

In [19]:
perform_and_evaluate_knn(3, distance_with_num_triplets_difference)

model evalutaion for k=3 and metric=<function distance_with_num_triplets_difference at 0x7f36978af940>
accuracy_score: 0.9983933899472114
[[   0    0    0    0    0    0    0    0    0    0    0    1    0    0
     0    0    0    0]
 [   0 1371    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0 1899    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    2    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0   34    0    1    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0   73    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0   22    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0   42    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0    2   32    0    0    0    0    0
     0    0    0    0]
 [   0 

In [20]:
perform_and_evaluate_knn(3, distance_with_num_nucleotides_difference)

model evalutaion for k=3 and metric=<function distance_with_num_nucleotides_difference at 0x7f36978afe50>
accuracy_score: 0.9986229056690383
[[   0    0    0    0    0    0    0    0    0    0    0    1    0    0
     0    0    0    0]
 [   0 1371    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0 1899    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    2    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0   34    0    0    0    0    0    0    0    0    1
     0    0    0    0]
 [   0    0    0    0    0   73    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0   22    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0   42    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0    2   32    0    0    0    0    0
     0    0    0    0]
 [  

In [21]:
#izdvojiti sars2 i uraditi klasifikaciju po who

<h1> KNN on who_type using p-adic distance </h1>

In [22]:
import numpy as np

virusTypes = np.array(X['Virus'])
X_who = []
indexes = []
for i in range(len(virusTypes)):
    if(virusTypes[i].strip() == "SARS_COV_2"):
        X_who.append([0, i])
        indexes.append(i)

sars2_only = X.filter(items=indexes, axis=0)

In [23]:
y_who=sars2_only["WHO_TYPE"].str.strip()

X_train, X_test, y_train, y_test = train_test_split(X_who, y_who, test_size=0.33, stratify=y_who)

In [24]:
y_train.unique()

array(['Delta', 'Eta', 'Beta', 'Omicron', 'Iota', 'Gamma', 'Epsilon',
       'Alpha', 'Lambda', 'Zeta', 'Kappa', 'Theta', 'Mu'], dtype=object)

In [25]:
perform_and_evaluate_knn(3, distance_from_matrix)

model evalutaion for k=3 and metric=<function distance_from_matrix at 0x7f36978af4c0>
accuracy_score: 0.19946524064171123
[[491   2   2   0   0   0   0   0   0   0   0   0   0]
 [169   2  20   1   0   0   0   0   1   0   0   0   0]
 [398   2  93   1   0   1   0   0   0   0   0   0   0]
 [405   0  31  59   0   0   0   0   0   0   0   0   0]
 [177   1   2   0  25   0   2   0   0   0   0   0   0]
 [405   2  32   3   0  53   0   0   0   0   0   0   0]
 [471   1  13   1   0   0   9   0   0   0   0   0   0]
 [ 41   0  11   0   0   0   0   0   0   0   0   0   0]
 [120   0   5   0   0   0  10   0   0   0   0   0   0]
 [ 17   0   5   0   0   0   0   0   0   0   0   0   0]
 [469   0   1   0   0   0  12   0   0   0  13   0   0]
 [  6   0   1   0   0   0   0   0   0   0   0   0   0]
 [141   0  12   0   0   0   0   0   0   0   0   0   1]]


In [26]:
perform_and_evaluate_knn(3, distance_with_num_triplets_difference)

model evalutaion for k=3 and metric=<function distance_with_num_triplets_difference at 0x7f36978af940>
accuracy_score: 0.3245989304812834
[[484   2   3   0   0   4   0   0   0   0   2   0   0]
 [168   2  21   1   0   0   0   0   1   0   0   0   0]
 [ 16   2 475   1   0   1   0   0   0   0   0   0   0]
 [ 13   0 423  59   0   0   0   0   0   0   0   0   0]
 [176   1   2   1  25   0   1   0   0   0   1   0   0]
 [400   2  32   0   0  60   0   0   0   0   1   0   0]
 [414   1  14   2   1   3  59   0   0   0   1   0   0]
 [  0   0  42  10   0   0   0   0   0   0   0   0   0]
 [103   0   5   0   0   0  10   0  17   0   0   0   0]
 [  2   0  18   2   0   0   0   0   0   0   0   0   0]
 [295   0  20   4   1 140   3   0   0   0  32   0   0]
 [  1   0   5   0   0   0   0   0   0   0   1   0   0]
 [  2   0 127  24   0   0   0   0   0   0   0   0   1]]


In [27]:
perform_and_evaluate_knn(3, distance_with_num_nucleotides_difference)

model evalutaion for k=3 and metric=<function distance_with_num_nucleotides_difference at 0x7f36978afe50>
accuracy_score: 0.3245989304812834
[[485   1   3   0   0   4   0   0   0   0   2   0   0]
 [168   2  21   1   0   0   0   0   1   0   0   0   0]
 [ 16   2 475   1   0   1   0   0   0   0   0   0   0]
 [ 13   0 422  59   0   0   0   0   0   1   0   0   0]
 [176   1   2   1  25   0   1   0   0   0   1   0   0]
 [400   2  32   0   0  59   0   0   0   0   2   0   0]
 [414   1  14   2   1   3  59   0   0   0   1   0   0]
 [  0   0  42  10   0   0   0   0   0   0   0   0   0]
 [103   0   5   0   0   0  10   0  17   0   0   0   0]
 [  1   0  18   3   0   0   0   0   0   0   0   0   0]
 [295   0  20   4   1 140   3   0   0   0  32   0   0]
 [  1   0   5   0   0   0   0   0   0   0   1   0   0]
 [  2   0 127  24   0   0   0   0   0   0   0   0   1]]


In [28]:
# TO DO zavrsiti implementaciju GridSearch (unakrsna validacija) i implementirati GridSearch za sve 3 klasifikacije

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
# gs = GridSearchCV(estimator=KNeighborsClassifier(), param_grid={'n_neighbors': list(range(2, 10)),
                                                               # 'metric' : [distance_from_matrix, distance_with_num_triplets_difference]})

In [31]:
# gs.fit(X_train, y_train)

In [32]:
# gs.best_score_

In [33]:
# y_pred = gs.predict(X_test)

In [34]:
# accuracy_score(y_test, y_pred)

In [35]:
# gs.best_estimator_